In [3]:
# Write your imports here
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

# Regression Models Lab
## Linear and logistic regression: theory and practice

In this lab you'll revisit and expand on your knowledge of modelling in general, as well as the fundamentals of linear and logistic regression. As a reminder, _linear regression_ is a regression model (regressor), and _logistic regression_ is a classification model (classifier).

This time, you'll use generated data, in order to separate some of the complexity of handling various datasets from inspecting and evaluating models.

**Use vectorization as much as possible!** You should be able to complete the lab using for-loops only to track the training steps.

### Problem 1. Generate some data for multiple linear regression (1 point)
As an expansion to the lecture, you'll create a dataset and a model.

Create a dataset of some (e.g., 50-500) observations of several (e.g., 5-20) independent features. You can use random generators for them; think about what distributions you'd like to use. Let's call them $x_1, x_2, ..., x_m$. The data matrix $X$ you should get should be of size $n \times m$. It's best if all features have different ranges.

Create the dependent variable by assigning coefficients $\bar{a_1}, \bar{a_2}, ..., \bar{a_m}, \bar{b}$ and calculating $y$ as a linear combination of the input features. Add some random noise to the functional values. I've used bars over coefficients to avoid confusion with the model parameters later.

Save the dataset ($X$ and $y$), and "forget" that the coefficients have ever existed. "All" you have is the file and the implicit assumption that there is a linear relationship between $X$ and $y$.

Lets create some random data around a multivariable linear function with given coefficients:

In [125]:
def create_random_data(coefficients, observations = 250):
    coefficients = coefficients.copy()
    
    X = np.column_stack([np.random.normal(random.randint(-10, 0), random.randint(1, 10), observations) for _ in range(len(coefficients) - 1)])
    b = coefficients.pop()
    coefficients_np = np.array(coefficients)

    y = X @ coefficients_np + b
    noise = np.random.normal(0, 2, observations)
    y_noisy = y + noise
    
    X_df = pd.DataFrame(X, columns=[f'Feature_{i+1}' for i in range(len(coefficients))])
    y_df = pd.DataFrame(y_noisy, columns = ["Target"])


    return pd.concat([X_df, y_df], axis = 1)


coefficients = [2, 3, 6, 12, -4, 10, 15, -45, 23, 0, 0, -1, 4]
observations = 250
X_y_df = create_random_data(coefficients, observations)
X_y_df.to_csv("dataset.csv", index = False)

### Problem 2. Check your assumption (1 point)
Read the dataset you just saved (this is just to simulate starting a new project). It's a good idea to test and verify our assumptions. Find a way to check whether there really is a linear relationship between the features and output.

Lets load the dataset:

In [126]:
dataset = pd.read_csv("dataset.csv")
dataset

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,Feature_11,Feature_12,Target
0,-12.222645,-12.862026,-19.812827,0.238868,-5.604076,-22.898432,-2.481869,-4.929514,-14.353434,-7.428352,-9.636395,-9.889862,-516.578259
1,-9.394832,-12.552980,-0.226421,-11.779376,2.776367,-5.620750,-10.775383,-0.822694,1.827162,1.264254,-9.339003,-10.862721,-334.662262
2,-23.560522,-10.177328,-1.695058,-0.141079,-0.975175,-9.778886,-0.006537,-9.288187,-17.119301,-17.433807,-8.944633,-7.398669,-145.928042
3,-12.528758,-15.591109,0.509936,-9.513235,2.881339,-9.629675,-13.231557,-5.446809,-11.747463,-12.235509,-11.760659,-6.090886,-500.332531
4,-0.934363,-12.489731,-1.770037,-12.668980,-7.852081,-11.631316,-5.097610,-7.662996,-20.176570,2.391676,-10.054706,-8.296786,-468.106297
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,-19.662860,-5.717533,-21.681519,-5.092920,-2.138577,7.795990,-7.498767,-1.573938,-9.577957,-4.139405,-12.051566,-6.595480,-412.738988
246,-9.309887,-8.067931,-3.731106,-1.195446,8.576007,-13.298540,6.074231,-9.112042,-7.010942,8.048932,-7.585141,-2.583402,96.817139
247,-13.754949,-14.812572,5.737743,2.993154,-9.083089,-15.360585,-7.774093,-4.787382,-8.265594,-4.571931,-9.500463,-6.778600,-199.749920
248,-18.351524,-11.582834,-9.268166,2.630443,-13.163066,-11.006400,-5.550712,-0.995019,-10.526060,-0.414012,-9.358448,0.086330,-427.811929


Now lets look at the correlation matrix:

In [130]:
dataset.corr()

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,Feature_11,Feature_12,Target
Feature_1,1.000000,0.021147,0.020833,-0.039436,0.031784,-0.007810,0.035215,-0.016989,0.073647,-0.124367,0.039514,0.068578,0.100719
Feature_2,0.021147,1.000000,-0.035045,0.008420,0.097120,0.059666,0.019581,0.017387,0.059117,-0.021705,-0.130923,0.071419,0.069178
Feature_3,0.020833,-0.035045,1.000000,0.076069,-0.064741,-0.112727,0.031387,-0.025983,-0.022892,-0.049652,-0.026250,-0.098229,0.156813
Feature_4,-0.039436,0.008420,0.076069,1.000000,-0.112572,-0.078279,-0.086949,0.005435,0.025748,-0.118807,0.093028,0.055094,0.187052
Feature_5,0.031784,0.097120,-0.064741,-0.112572,1.000000,0.092035,0.036878,0.048808,-0.037603,-0.046233,0.051943,-0.003882,-0.119550
Feature_6,-0.007810,0.059666,-0.112727,-0.078279,0.092035,1.000000,0.003534,-0.043323,-0.030455,-0.133640,-0.041184,0.064644,0.212975
Feature_7,0.035215,0.019581,0.031387,-0.086949,0.036878,0.003534,1.000000,0.014692,0.022549,0.005856,0.010937,0.074959,0.154146
Feature_8,-0.016989,0.017387,-0.025983,0.005435,0.048808,-0.043323,0.014692,1.000000,-0.058207,0.083921,0.068674,-0.050956,-0.767345
Feature_9,0.073647,0.059117,-0.022892,0.025748,-0.037603,-0.030455,0.022549,-0.058207,1.000000,0.005898,0.015118,0.018788,0.566888
Feature_10,-0.124367,-0.021705,-0.049652,-0.118807,-0.046233,-0.133640,0.005856,0.083921,0.005898,1.000000,0.032905,-0.080652,-0.121940


We can see, that the linear relationship is not very apparent.

### Problem 3. Figure out the modelling function (1 point)
The modelling function for linear regression is of the form
$$ \tilde{y} = \sum_{i=1}^{m}a_i x_i + b $$

If you want to be clever, you can find a way to represent $b$ in the same way as the other coefficients.

Write a Python function which accepts coefficients and data, and ensure (test) it works correctly.

### Problem 4. Write the cost function and compute its gradients (1 point)
Use MSE as the cost function $J$. Find a way to compute, calculate, or derive its gradients w.r.t. the model parameters $a_1, ..., a_m, b$

Note that computing the cost function value and its gradients are two separate operations. Quick reminder: use vectorization to compute all gradients (maybe with the exception of $\frac{\partial J}{\partial b}$) at the same time.

### Problem 5. Perform gradient descent (1 point)
Perform weight updates iteratively. Find a useful criterion for stopping. For most cases, just using a fixed (large) number of steps is enough.

You'll need to set a starting point (think about which one should be good, and how it matters); and a learning rate.

### Problem 6. Do other cost functions work? (2 points)
Repeat the process in problems 4 and 5 with MAE, and then again - with the [Huber loss](https://en.wikipedia.org/wiki/Huber_loss). Both of them are less sensitive to outliers / anomalies than MSE); with the Huber loss function being specifically made for datasets with outliers.

Explain your findings. Is there a cost function that works much better? How about speed of training (measured in wall time)?

### Problem 7. Experiment with the learning rate (1 point)
Use your favorite cost function. Run several "experiments" with different learning rates. Try really small, and really large values. Observe and document your findings.

### Problem 8. Generate some data for classification (1 point)
You'll need to create two clusters of points (one cluster for each class). I recomment using `scikit-learn`'s `make_blobs()` ([info](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_blobs.html)). Use as many features as you used in problem 1.

### Problem 9. Perform logistic regression (1 point)
Reuse the code you wrote in problems 3-7 as much as possible. If you wrote vectorized functions with variable parameters - you should find this easy. If not - it's not too late to go back and refactor your code.

The modelling function for logistic regression is
$$ \tilde{y} = \frac{1}{1+\exp{(-\sum_{i=1}^{m}a_i x_i + b)}}$$. Find a way to represent it using as much of your previous code as you can.

The most commonly used loss function is the [cross-entropy](https://en.wikipedia.org/wiki/Cross-entropy).

Experiment with different learning rates, basically repeating what you did in problem 7.

### * Problem 10. Continue experimenting and delving deep into ML
You just saw how modelling works and how to implement some code. Some of the things you can think about (and I recommend you pause and ponder on some of them are):
* Code: OOP can be your friend sometimes. `scikit-learn`'s models have `fit()`, `predict()` and `score()` methods.
* Data: What approaches work on non-generated data?
* Evaluation: How well do different models (and their "settings" - hyperparameters) actually work in practice? How do we evaluate a model in a meaningful way?
* Optimization - maths: Look at what `optimizers` (or solvers) are used in `scikit-learn` and why. Many "tricks" revolve around making the algorithm converge (finish) in fewer iterations, or making it more numerically stable.
* Optimization - code: Are there ways to make the code run fastr?